# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
IMG_DIR = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
VAL_PCT = 10
VAL_FORMAT = 'png'

In [5]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')

In [6]:
os.getcwd()

'C:\\Users\\Khalid\\Documents\\projects\\Generatives\\Generative_Models'

In [7]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [8]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [9]:
inputs_shape= IMG_SIZE=(200, 200, 3)
BATCH_SIZE = 32
latent_dim = 32
intermediate_dim = 16

In [10]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [11]:
import tensorflow as tf

In [12]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

In [13]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [14]:
_instance_scale

1.0

In [15]:
import numpy as np
from collections.abc import Iterable

In [16]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [17]:
_outputs_shape

120000

# Model's Layers definition

In [18]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

dec_lays = [tf.keras.layers.Dense(units=latent_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [19]:
model_name = dataset_name+'ITAAEpsnr'
recoding_dir='recoding'+sep_local+model_name

In [20]:
recoding_dir

'recoding\\pokemonIAAEpsnr'

In [21]:
from training.inputs_adversarial.autoencoders.AAE import AAE as AE

In [22]:
inputs_shape=IMG_SIZE

In [23]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_lays#enc_lays2#
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays#dec_lays2#
    }
]

In [24]:
from utils.data_and_files.file_utils import create_if_not_exist

In [25]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [26]:
create_if_not_exist(_restore)
_restore

'recoding\\pokemonIAAEpsnr\\var_save_dir'

In [27]:
#to restore trained model, set filepath=_restore

In [ ]:
from stats.traditional_adversarial_losses import \
    create_inputs_adversarial_real_losses, \
    create_inputs_adversarial_fake_losses, \
    create_inputs_adversarial_losses

inputs_adversarial_losses = {
    'inputs_adversarial_losses': create_inputs_adversarial_losses,
    'inputs_adversarial_real_losses': create_inputs_adversarial_real_losses,
    'inputs_adversarial_fake_losses': create_inputs_adversarial_fake_losses,
}

In [28]:
ae = AE( 
    name=model_name,
    inputs_adversarial_losses=inputs_adversarial_losses,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 200, 200, 16)      64        
_________________________________________________________________
flatten (Flatten)            (None, 640000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                20480032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
activity_regularization (Act (None, 32)                0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 120000)            2040000   
_________________________________________________________________
reshape (Reshape)            (None, 200, 200, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 200, 200, 3)       12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 200, 3)       0

  WARNING  | None


In [29]:
from evaluation.quantitive_metrics.psnr import psnr
#from evaluation.quantitive_metrics.ssmi import ssmi
#from evaluation.quantitive_metrics.sharp_diff import sharp_diff


In [30]:
ae.compile(loss={'x_logits': psnr})

Model: "pokemonIAAEpsnr"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
inference (Model)            (None, 32)                20480224  
_________________________________________________________________
generative (Model)           (None, 200, 200, 3)       2041596   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 200, 200, 3)]     0         
Total params: 22,521,820
Trainable params: 22,521,750
Non-trainable params: 70
_________________________________________________________________
None


# Callbacks

In [31]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [32]:
progbar = NotebookPrograssBar(leave_outer=False)

In [33]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [34]:
_restore

'recoding\\pokemonIAAEpsnr\\var_save_dir'

In [35]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [36]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'recoding\\pokemonIAAEpsnr\\csv_dir\\pokemonIAAEpsnr.csv'

In [37]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [38]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [39]:
import numpy as np

# Model Training

In [40]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=100,
    epochs=int(1e6), 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=100
)


training traditional basicAE
Train for 10 steps, validate for 10 steps


Epoch 1/10
10/10 [==============================] - ETA: 31s - loss: 7.3825 - psnr: 7.3397 - ssmi: 0.0342 - sharp_diff: 7.311 - ETA: 14s - loss: 7.3511 - psnr: 7.3056 - ssmi: 0.0320 - sharp_diff: 7.192 - ETA: 8s - loss: 7.3185 - psnr: 7.2680 - ssmi: 0.0311 - sharp_diff: 7.114 - ETA: 5s - loss: 7.2827 - psnr: 7.2283 - ssmi: 0.0309 - sharp_diff: 7.07 - ETA: 4s - loss: 7.2614 - psnr: 7.2041 - ssmi: 0.0316 - sharp_diff: 7.02 - ETA: 2s - loss: 7.2539 - psnr: 7.1918 - ssmi: 0.0320 - sharp_diff: 6.98 - ETA: 1s - loss: 7.2365 - psnr: 7.1711 - ssmi: 0.0306 - sharp_diff: 6.89 - ETA: 1s - loss: 7.2381 - psnr: 7.1698 - ssmi: 0.0306 - sharp_diff: 6.85 - ETA: 0s - loss: 7.2278 - psnr: 7.1565 - ssmi: 0.0292 - sharp_diff: 6.74 - 7s 750ms/step - loss: 7.2183 - psnr: 7.1441 - ssmi: 0.0284 - sharp_diff: 6.6592 - val_loss: 7.5106 - val_psnr: 7.5059 - val_ssmi: 0.1296 - val_sharp_diff: 11.4406


Epoch 2/10
10/10 [==============================] - ETA: 1s - loss: 7.0770 - psnr: 6.9772 - ssmi: 0.0185 - sharp_diff: 5.87 - ETA: 1s - loss: 7.0058 - psnr: 6.9050 - ssmi: 0.0187 - sharp_diff: 5.84 - ETA: 1s - loss: 6.9851 - psnr: 6.8847 - ssmi: 0.0193 - sharp_diff: 5.89 - ETA: 0s - loss: 6.9857 - psnr: 6.8848 - ssmi: 0.0200 - sharp_diff: 5.90 - ETA: 0s - loss: 6.9532 - psnr: 6.8514 - ssmi: 0.0199 - sharp_diff: 5.92 - ETA: 0s - loss: 6.9501 - psnr: 6.8473 - ssmi: 0.0213 - sharp_diff: 5.90 - ETA: 0s - loss: 6.9270 - psnr: 6.8230 - ssmi: 0.0209 - sharp_diff: 5.85 - ETA: 0s - loss: 6.9254 - psnr: 6.8203 - ssmi: 0.0218 - sharp_diff: 5.88 - ETA: 0s - loss: 6.9233 - psnr: 6.8174 - ssmi: 0.0222 - sharp_diff: 5.89 - 3s 280ms/step - loss: 6.9161 - psnr: 6.8085 - ssmi: 0.0223 - sharp_diff: 5.8759 - val_loss: 7.4516 - val_psnr: 7.4397 - val_ssmi: 0.0971 - val_sharp_diff: 10.4788


Epoch 3/10
10/10 [==============================] - ETA: 1s - loss: 6.6904 - psnr: 6.5700 - ssmi: 0.0192 - sharp_diff: 5.48 - ETA: 1s - loss: 6.7392 - psnr: 6.6190 - ssmi: 0.0204 - sharp_diff: 5.62 - ETA: 1s - loss: 6.5724 - psnr: 6.4512 - ssmi: 0.0185 - sharp_diff: 5.42 - ETA: 0s - loss: 6.6236 - psnr: 6.5009 - ssmi: 0.0189 - sharp_diff: 5.48 - ETA: 0s - loss: 6.5775 - psnr: 6.4532 - ssmi: 0.0186 - sharp_diff: 5.48 - ETA: 0s - loss: 6.5719 - psnr: 6.4458 - ssmi: 0.0186 - sharp_diff: 5.47 - ETA: 0s - loss: 6.5928 - psnr: 6.4662 - ssmi: 0.0190 - sharp_diff: 5.50 - ETA: 0s - loss: 6.5757 - psnr: 6.4475 - ssmi: 0.0185 - sharp_diff: 5.48 - ETA: 0s - loss: 6.5702 - psnr: 6.4400 - ssmi: 0.0189 - sharp_diff: 5.50 - 3s 273ms/step - loss: 6.5456 - psnr: 6.4143 - ssmi: 0.0189 - sharp_diff: 5.5155 - val_loss: 7.3401 - val_psnr: 7.3230 - val_ssmi: 0.0873 - val_sharp_diff: 10.3230


Epoch 4/10
10/10 [==============================] - ETA: 1s - loss: 6.2838 - psnr: 6.1303 - ssmi: 0.0164 - sharp_diff: 5.28 - ETA: 1s - loss: 6.2808 - psnr: 6.1303 - ssmi: 0.0158 - sharp_diff: 5.34 - ETA: 1s - loss: 6.2907 - psnr: 6.1392 - ssmi: 0.0159 - sharp_diff: 5.41 - ETA: 0s - loss: 6.2433 - psnr: 6.0902 - ssmi: 0.0151 - sharp_diff: 5.40 - ETA: 0s - loss: 6.2231 - psnr: 6.0680 - ssmi: 0.0156 - sharp_diff: 5.43 - ETA: 0s - loss: 6.1893 - psnr: 6.0324 - ssmi: 0.0155 - sharp_diff: 5.45 - ETA: 0s - loss: 6.1911 - psnr: 6.0333 - ssmi: 0.0160 - sharp_diff: 5.46 - ETA: 0s - loss: 6.1703 - psnr: 6.0115 - ssmi: 0.0159 - sharp_diff: 5.45 - ETA: 0s - loss: 6.1555 - psnr: 5.9946 - ssmi: 0.0160 - sharp_diff: 5.46 - 3s 273ms/step - loss: 6.1454 - psnr: 5.9827 - ssmi: 0.0161 - sharp_diff: 5.4801 - val_loss: 7.3068 - val_psnr: 7.2840 - val_ssmi: 0.0950 - val_sharp_diff: 10.4673


...

Epoch 6/10
 9/10 [==========================>...] - ETA: 2s - loss: 46819.0625 - inputs_fake_discriminator_outputs_loss: 0.1976 - inputs_real_discriminator_outputs_loss: 0.3093 - x_logits_loss: 46818.3711 - x_logits_psnr: 8.7802 - x_logits_ssmi: 0.0428 - x_logits_sharp_diff: 7.198 - ETA: 2s - loss: 47113.2305 - inputs_fake_discriminator_outputs_loss: 0.2005 - inputs_real_discriminator_outputs_loss: 0.3180 - x_logits_loss: 47112.5312 - x_logits_psnr: 8.7599 - x_logits_ssmi: 0.0408 - x_logits_sharp_diff: 7.262 - ETA: 2s - loss: 48355.9831 - inputs_fake_discriminator_outputs_loss: 0.1963 - inputs_real_discriminator_outputs_loss: 0.3324 - x_logits_loss: 48355.2695 - x_logits_psnr: 8.6653 - x_logits_ssmi: 0.0386 - x_logits_sharp_diff: 7.284 - ETA: 1s - loss: 49150.8076 - inputs_fake_discriminator_outputs_loss: 0.1916 - inputs_real_discriminator_outputs_loss: 0.3437 - x_logits_loss: 49150.0859 - x_logits_psnr: 8.5927 - x_logits_ssmi: 0.0372 - x_logits_sharp_diff: 7.298 - ETA: 1s - loss: 4887

Epoch 7/10
10/10 [==============================] - ETA: 3s - loss: 48206.1719 - inputs_fake_discriminator_outputs_loss: 0.1850 - inputs_real_discriminator_outputs_loss: 0.3143 - x_logits_loss: 48205.4844 - x_logits_psnr: 8.6921 - x_logits_ssmi: 0.0396 - x_logits_sharp_diff: 7.356 - ETA: 2s - loss: 47353.4043 - inputs_fake_discriminator_outputs_loss: 0.1894 - inputs_real_discriminator_outputs_loss: 0.3000 - x_logits_loss: 47352.7266 - x_logits_psnr: 8.7796 - x_logits_ssmi: 0.0407 - x_logits_sharp_diff: 7.284 - ETA: 2s - loss: 46717.9674 - inputs_fake_discriminator_outputs_loss: 0.1859 - inputs_real_discriminator_outputs_loss: 0.2916 - x_logits_loss: 46717.3008 - x_logits_psnr: 8.8361 - x_logits_ssmi: 0.0419 - x_logits_sharp_diff: 7.275 - ETA: 2s - loss: 47609.2949 - inputs_fake_discriminator_outputs_loss: 0.1835 - inputs_real_discriminator_outputs_loss: 0.2921 - x_logits_loss: 47608.6328 - x_logits_psnr: 8.7460 - x_logits_ssmi: 0.0396 - x_logits_sharp_diff: 7.267 - ETA: 1s - loss: 4794

Epoch 8/10
10/10 [==============================] - ETA: 2s - loss: 60465.1445 - inputs_fake_discriminator_outputs_loss: 0.1703 - inputs_real_discriminator_outputs_loss: 0.2840 - x_logits_loss: 60464.5078 - x_logits_psnr: 7.7133 - x_logits_ssmi: 0.0154 - x_logits_sharp_diff: 7.361 - ETA: 2s - loss: 54691.2754 - inputs_fake_discriminator_outputs_loss: 0.1719 - inputs_real_discriminator_outputs_loss: 0.2782 - x_logits_loss: 54690.6406 - x_logits_psnr: 8.1655 - x_logits_ssmi: 0.0279 - x_logits_sharp_diff: 7.336 - ETA: 2s - loss: 53134.3477 - inputs_fake_discriminator_outputs_loss: 0.1722 - inputs_real_discriminator_outputs_loss: 0.2841 - x_logits_loss: 53133.7070 - x_logits_psnr: 8.2979 - x_logits_ssmi: 0.0298 - x_logits_sharp_diff: 7.323 - ETA: 2s - loss: 51547.3516 - inputs_fake_discriminator_outputs_loss: 0.1733 - inputs_real_discriminator_outputs_loss: 0.2833 - x_logits_loss: 51546.7109 - x_logits_psnr: 8.4366 - x_logits_ssmi: 0.0314 - x_logits_sharp_diff: 7.283 - ETA: 1s - loss: 5041

Epoch 9/10
10/10 [==============================] - ETA: 3s - loss: 46452.2695 - inputs_fake_discriminator_outputs_loss: 0.1797 - inputs_real_discriminator_outputs_loss: 0.2707 - x_logits_loss: 46451.6406 - x_logits_psnr: 8.8863 - x_logits_ssmi: 0.0398 - x_logits_sharp_diff: 7.238 - ETA: 2s - loss: 46484.1953 - inputs_fake_discriminator_outputs_loss: 0.1720 - inputs_real_discriminator_outputs_loss: 0.2815 - x_logits_loss: 46483.5625 - x_logits_psnr: 8.8759 - x_logits_ssmi: 0.0405 - x_logits_sharp_diff: 7.200 - ETA: 2s - loss: 46769.5182 - inputs_fake_discriminator_outputs_loss: 0.1684 - inputs_real_discriminator_outputs_loss: 0.2805 - x_logits_loss: 46768.8867 - x_logits_psnr: 8.8492 - x_logits_ssmi: 0.0416 - x_logits_sharp_diff: 7.215 - ETA: 2s - loss: 46978.4326 - inputs_fake_discriminator_outputs_loss: 0.1671 - inputs_real_discriminator_outputs_loss: 0.2837 - x_logits_loss: 46977.7969 - x_logits_psnr: 8.8316 - x_logits_ssmi: 0.0421 - x_logits_sharp_diff: 7.245 - ETA: 1s - loss: 4659

Epoch 10/10
10/10 [==============================] - ETA: 2s - loss: 47817.8438 - inputs_fake_discriminator_outputs_loss: 0.1526 - inputs_real_discriminator_outputs_loss: 0.2315 - x_logits_loss: 47817.2734 - x_logits_psnr: 8.7202 - x_logits_ssmi: 0.0356 - x_logits_sharp_diff: 7.266 - ETA: 2s - loss: 48215.5410 - inputs_fake_discriminator_outputs_loss: 0.1525 - inputs_real_discriminator_outputs_loss: 0.2420 - x_logits_loss: 48214.9609 - x_logits_psnr: 8.6872 - x_logits_ssmi: 0.0356 - x_logits_sharp_diff: 7.268 - ETA: 2s - loss: 48617.7161 - inputs_fake_discriminator_outputs_loss: 0.1495 - inputs_real_discriminator_outputs_loss: 0.2431 - x_logits_loss: 48617.1367 - x_logits_psnr: 8.6563 - x_logits_ssmi: 0.0354 - x_logits_sharp_diff: 7.261 - ETA: 2s - loss: 47809.8584 - inputs_fake_discriminator_outputs_loss: 0.1501 - inputs_real_discriminator_outputs_loss: 0.2504 - x_logits_loss: 47809.2695 - x_logits_psnr: 8.7409 - x_logits_ssmi: 0.0361 - x_logits_sharp_diff: 7.233 - ETA: 1s - loss: 460